# Simple Vector Store

#### Load documents, build the GPTVectorStoreIndex

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader('../../../examples/paul_graham_essay/data').load_data()

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents)

In [ ]:
# save index to disk
index.set_index_id("vector_index")
index.storage_context.persist('./storage')

In [ ]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='storage')
# load index
index = load_index_from_storage(storage_context, index_id="vector_index")

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

**Query Index with SVM/Linear Regression**

Use Karpathy's [SVM-based](https://twitter.com/karpathy/status/1647025230546886658?s=20) approach. Set query as positive example, all other datapoints as negative examples, and then fit a hyperplane.

In [ ]:
query_modes = [
    "svm",
    "linear_regression",
    "logistic_regression",
]
for query_mode in query_modes:
# set Logging to DEBUG for more detailed outputs
    query_engine = index.as_query_engine(
        vector_store_query_mode=query_mode
    )
    response = query_engine.query(
        "What did the author do growing up?"
    )
    print(f"Query mode: {query_mode}")
    display(Markdown(f"<b>{response}</b>"))

In [ ]:
display(Markdown(f"<b>{response}</b>"))

In [ ]:
print(response.source_nodes[0].source_text)

**Query Index with custom embedding string**

In [ ]:
from llama_index.indices.query.schema import QueryBundle

In [ ]:
query_bundle = QueryBundle(
    query_str="What did the author do growing up?", 
    custom_embedding_strs=['The author grew up painting.']
)
query_engine = index.as_query_engine()
response = query_engine.query(query_bundle)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

#### Get Sources

In [ ]:
print(response.get_formatted_sources())

#### Query Index with LlamaLogger

Log intermediate outputs and view/use them.

In [ ]:
from llama_index.logger import LlamaLogger
from llama_index import ServiceContext

llama_logger = LlamaLogger()
service_context = ServiceContext.from_defaults(llama_logger=llama_logger)

In [ ]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=2,
    # response_mode="tree_summarize"
)
response = query_engine.query(
    "What did the author do growing up?",
)

In [ ]:
# get logs
service_context.llama_logger.get_logs()